In [1]:
import pickle
import os
import csv
import numpy as np
# Specify the working directory
os.chdir('/Users/david/Desktop/FinetuneEmbed')

from mod.mod_text import *
from mod.options import *

/Users/david/anaconda3/envs/FinetuneBERT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description="Initialize arguments for fine-tuning a model.")

parser.add_argument("--model_name", type=str, default="GIST-small-Embedding-v0", help="Name of the chosen model.")
parser.add_argument("--random_states", type=list, default=list(range(41, 51)), help="Random indices used to generate data.")

# Arguments for fine-tuning
parser.add_argument("--evaluation_strategy", type=str, default="epoch", help="The evaluation strategy used in training.")
parser.add_argument("--save_strategy", type=str, default="epoch", help="The checkpoint saving strategy used in training.")
parser.add_argument("--learning_rate", type=float, default=1e-5, help="The learning rate used in training.")
parser.add_argument("--num_train_epochs", type=int, default=20, help="The number of epoches used in training.")
parser.add_argument("--max_grad_norm", type=float, default=0.7, help="The maximum gradient norm used in training.")
parser.add_argument("--warmup_ratio", type=float, default=0.1, help="The warm up ratio used in training.")
parser.add_argument("--weight_decay", type=float, default=0.2, help="The gradient weight decay used in training.")

# Arguments for best model final tuning
parser.add_argument("--final_evaluation_strategy", type=str, default="no", help="The evaluation strategy used in final training.")
parser.add_argument("--final_save_strategy", type=str, default="no", help="The checkpoint saving strategy used in final training.")
parser.add_argument("--final_learning_rate", type=float, default=1e-6, help="The learning rate used in final training.")
parser.add_argument("--final_num_train_epochs", type=int, default=10, help="The number of epoches used in final training.")
parser.add_argument("--final_max_grad_norm", type=float, default=0.7, help="The maximum gradient norm used in final training.")
parser.add_argument("--final_weight_decay", type=float, default=0.2, help="The gradient weight decay used in final training.")

_StoreAction(option_strings=['--final_weight_decay'], dest='final_weight_decay', nargs=None, const=None, default=0.2, type=<class 'float'>, choices=None, required=False, help='The gradient weight decay used in final training.', metavar=None)

In [ ]:

data_dir = "./data/long_vs_shortTF"
save_csv_dir = "./res/2024_1209/NoInstruct-small-Embedding-v0/LongShortTF/long_vs_shortTF_finetune_auc.csv"
output_path = "./res/2024_1209/NoInstruct-small-Embedding-v0/LongShortTF/LongShortTF_model_"
model_name = 'avsolatorio/NoInstruct-small-Embedding-v0'


random_state = 41
output_dir = output_path  + str(random_state)

In [3]:
train_dir = data_dir + "/TrainEvalTestData/train_data_" + str(random_state) + ".pkl"
eva_dir = data_dir + "/TrainEvalTestData/eval_data_" + str(random_state) + ".pkl"
test_dir = data_dir + "/TrainEvalTestData/test_data_" + str(random_state) + ".pkl"

# prepare the input data
with open(train_dir, "rb") as f:
    train_data = pickle.load(f)
with open(eva_dir, "rb") as f:
    eval_data = pickle.load(f)
with open(test_dir, "rb") as f:
    test_data = pickle.load(f)

In [6]:
len(eval_data['genes'])

18

In [7]:
train_texts, train_labels = train_data['desc'], train_data['labels']
eval_texts, eval_labels = eval_data['desc'], eval_data['labels']
test_texts, test_labels = test_data['desc'], test_data['labels']

In [8]:
len(eval_texts), len(eval_labels)

(18, 18)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
train_dataset = TextDataset(train_texts, train_labels, tokenizer)
val_dataset = TextDataset(eval_texts, eval_labels, tokenizer)

In [ ]:
mod_dir, val_auc = one_fold_training(train_texts, train_labels, eval_texts, eval_labels, 
                                        tokenizer, output_dir, 0, model_name, args)

18